In [7]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chat = ChatOpenAI(temperature=0.1)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

In [8]:
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

Chat model 은 대화에 최적화 되어 있는데 단지 질문을 받을 뿐 아니라, 대화를 할 수 있다는 뜻이다.

대화는, 그룹 오브 메세지스 즉, 여러 메세지의 묶음이라는 의미이다.

상대로서 대화의 맥락에 맞게 대답할 수 있다는 뜻이다. - Chat Model



만약 model 의 설정을 바꾸고 싶다면 

In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?"
    ),
]

SystemMessage 로 AI 에 일종의 기본 설정, 기본 값, 기본 context 설정을 함.

AIMessage 로 string 을 미리 만들어두고 일종의 가상 대화(마치 AI 가 답변한 것처럼)를 만들었음.

HumanMessage 로 사용자로서 질문을 함.

In [4]:
chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

predict 와 predict_messages 는 string(text) 과 messages 를 predict 하는 차이점이 있다.

---

# Prompt Template 에 대해 알아보자



prompt 는 중요하다. 왜? 우리가 LLM과 의사소통할 수 있는 유일한 방법이기 때문에.

prompt 성능이 좋으면 llm 의 성능도 좋을 것이다.

prompt template 의 종류는 두 가지로 볼 수 있다.
1. 일반 prompt template - template 를 string 을 이용해서 만든다.
2. chat prompt template - template 를 message 로 부터 만든다.

## normal prompt template

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thiland")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

## Chat Prompt Template

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# chat 을 상단에 위치
chat = ChatOpenAI(temperature=0.1)
# template 로 ChatPromptTemplate 의 from_messages 사용 - 리스트를 넘겨준다.
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are geography expert. And you only reply in {language}."),
        ("ai", "안녕, 내 이름은 {name}이야!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="Kim",
    country_a="Japan",
    country_b="China",
)

chat.predict_messages(prompt)

AIMessage(content='일본과 중국 사이의 거리는 대략 1500km입니다. 제 이름은 Kim입니다.')

# OutputParser 와 LCEL(표현언어)

lcel - 다양한 template 와 llm 호출, 그리고 서로 다른 응답(response)을 함께 사용하게 해준다.

outputparser 가 필요한 이유?

llm 의 응답을 변형해야 할 때가 있기 때문.

llm 은 항상 텍스트로 대답하기 때문에 그 응답을 무언가로 변형 시키고 싶은 경우가 생긴다.

In [13]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. everything you are asked will be answered with a comma seperate list of max {max_items} in lower case. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?",
)

chat.predict_messages(prompt)

AIMessage(content='mercury, venus, earth, mars, jupiter, saturn, uranus, neptune, pluto')

In [19]:
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

# Chain 
chain - 기본적으로 모든 요소를 합쳐주는 역할을 한다.

합쳐진 요소들은 하나의 chain 으로 실행된다. 하나하나 순서대로 result 를 반환할때까지.

chain 은 '|' 주변에 입력한 값들로 chain 을 형성한다. - LangChaing 의 핵심.

In [20]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

## Chain 은 Chain 끼리 결합할 수 있다.

### 2개의 체인 결합
chain_one = template | chat | CommaOuputParser()

chain_two = template_2 | chat | OutputParser()

### all = chain_one | chain_two | outputparser()

---


# LECL 에 대해 더 알아보자.

expression language 의 구성요소(component) 부터 살펴보자.

chain 이 가질 수 있는 구성요소(component)의 목록
1. Prompt
2. Retriever
3. LLM, ChatModel
4. Tool
5. OutputParser

각 구성요소는 input type 과 output type 을 갖는다.

```python 
chain = template | chat | outputparser()

chain.invoke({
    "max_items": 5,
    "question": "what are the pokemons?",
})
```

prompt 의 input type 은 dictionary 이고 prompt template 에 보내진다.  
prompt의 output type 은 prompt value 이다.

다음으로 prompt value 는 chat 으로 이동한다.  
chat model 의 input type 은 list of chat messages or a PromptValue 이다.  
chat model 의 output type 은 ChatMessage 이다.

그리고 그 값은 outputparser 에게 전해진다.
outputparser 의 input type 은 LLM 의 출력값 또는 ChatModel 이다.
outputparser 의 output type 은 

최종 출력값은 chain 에 입력한 코드가 끝날때까지 옆의 component 에 전달되고,  
사용자는 최종 출력값만 전달받는다.

chain 끼리의 결합을 살펴보자.

chef_chain

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_template = ChatPromptTemplate.from_messages(
    [
        # system: 역할
        (
            "system",
            "you are a world-class international chef. you can create easy to follow recipies for any type of duisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_template | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        # system: 역할
        (
            "system",
            "you are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. you don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain

result = final_chain.invoke({"cuisine": "indian"})

content="As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you an alternative version of Chicken Tikka Masala using paneer instead of chicken. Paneer is a popular Indian cheese that is a great substitute for meat in many dishes. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground turmeric\n- 1 teaspoon garam masala\n- 1 teaspoon ground coriander\n- 1 teaspoon chili powder (adjust to taste)\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup heavy cream\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the paneer pieces in the yogurt and spice mixt

In [24]:
print(result.content)

As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you an alternative version of Chicken Tikka Masala using paneer instead of chicken. Paneer is a popular Indian cheese that is a great substitute for meat in many dishes. Here's how you can modify the recipe:

Ingredients:
- 1 lb paneer, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the paneer pieces in the yogurt and spice mixture. Refrigerate for at least 1

# Homework 1
- 프로그래밍 언어에 대한 시를 쓰는 데 특화된 체인과 시를 설명하는 데 특화된 체인을 만드세요.
- LCEL을 사용해 두 체인을 서로 연결합니다.
- 최종 체인은 프로그래밍 언어의 이름을 받고 시와 그 설명으로 응답해야 합니다.
- 모델로는 "gpt-3.5-turbo"를 사용하고 프롬프트에는 `ChatPromptTemplate`을 사용하세요.

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# poet chain
poet_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 프로그래밍 언어에 대한 시를 쓰는 전문가야, 내가 프로그래밍언어를 주면 그 언어의 특징이나 사람들이 좋아하는 이유에 맞게 재밌는 시를 지어서 주면 돼. 너무 길지 않도록 시를 잘 지어줘.",
        ),
        ("human", "{program_language}"),
    ]
)

poet_chain = poet_template | chat

# explain chain
poem_explain_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 시를 읽고 평범한 사람들이 쉽게 이해할 수 있도록 설명하는데 전문가야. 시를 보여주면 너는 아주 쉽게 설명해주면 좋겠어. 만약 설명할 수 없다면 글을 만들지 말고 설명할 수 없다고 얘기해줘.",
        ),
        ("human", "{poem}"),
    ]
)

poem_explain_chain = poem_explain_template | chat

all_chain = {"poem": poet_chain} | poem_explain_chain

result = all_chain.invoke({"program_language": "python"})

result

파이썬이란 이름은 뱀처럼 매끈해  
코드 한 줄 한 줄 읽기 쉽게 만들어  

들여쓰기로 블록을 구분하고  
간결한 문법으로 사람들을 사로잡아  

라이브러리가 풍부해 문제 해결이 쉬워  
데이터 분석, 웹 개발 모두 가능해  

파이썬은 마치 마법 같아  
그 매력에 빠져 빠져 또 빠져이 시는 파이썬 프로그래밍 언어에 대한 찬사를 담은 시입니다. 시인은 파이썬이 매끈하고 읽기 쉽다고 표현하며, 들여쓰기와 간결한 문법으로 코드를 이해하기 쉽게 만든다고 말하고 있습니다. 또한 파이썬의 풍부한 라이브러리로 인해 다양한 문제를 해결할 수 있고, 데이터 분석이나 웹 개발과 같은 다양한 분야에서 활용할 수 있다고 언급하고 있습니다. 마지막으로 시인은 파이썬의 매력에 빠져들어 마법 같다고 묘사하며 계속해서 파이썬에 매료되어간다는 느낌을 표현하고 있습니다.

AIMessageChunk(content='이 시는 파이썬 프로그래밍 언어에 대한 찬사를 담은 시입니다. 시인은 파이썬이 매끈하고 읽기 쉽다고 표현하며, 들여쓰기와 간결한 문법으로 코드를 이해하기 쉽게 만든다고 말하고 있습니다. 또한 파이썬의 풍부한 라이브러리로 인해 다양한 문제를 해결할 수 있고, 데이터 분석이나 웹 개발과 같은 다양한 분야에서 활용할 수 있다고 언급하고 있습니다. 마지막으로 시인은 파이썬의 매력에 빠져들어 마법 같다고 묘사하며 계속해서 파이썬에 매료되어간다는 느낌을 표현하고 있습니다.')

In [29]:
print(result.content)

이 시는 파이썬 프로그래밍 언어에 대한 찬사를 담은 시입니다. 시인은 파이썬이 매끈하고 읽기 쉽다고 표현하며, 들여쓰기와 간결한 문법으로 코드를 이해하기 쉽게 만든다고 말하고 있습니다. 또한 파이썬의 풍부한 라이브러리로 인해 다양한 문제를 해결할 수 있고, 데이터 분석이나 웹 개발과 같은 다양한 분야에서 활용할 수 있다고 언급하고 있습니다. 마지막으로 시인은 파이썬의 매력에 빠져들어 마법 같다고 묘사하며 계속해서 파이썬에 매료되어간다는 느낌을 표현하고 있습니다.
